In [258]:
import numpy as np
import scipy as sp

In [183]:
f= open('voc', 'r')
voc = f.read().strip().split("\n")

6806

In [63]:
def parse_vocabulary(vocab):
    type_to_index = {}
    index_to_type = {}
    for word in set(vocab):
        index_to_type[len(index_to_type)] = word
        type_to_index[word] = len(type_to_index)        
    return (type_to_index, index_to_type)

In [65]:
type_to_index = parse_vocabulary(voc)[0]
index_to_type = parse_vocabulary(voc)[1]

In [70]:
ff = open('doc', 'r')
doc = ff.read().strip().split("\n")

2221

In [201]:
def parse_data(corpus):
        word_ids = [];
        word_cts = [];
        
        for document_line in corpus:
            document_word_dict = {}
            for token in document_line.split():
                if token not in type_to_index:
                    continue
                
                type_id = type_to_index[token]
                if type_id not in document_word_dict:
                    document_word_dict[type_id] = 0
                document_word_dict[type_id] += 1
            
            word_ids.append(np.array(list(document_word_dict.keys())))
            word_cts.append(np.array(list(document_word_dict.values()))[numpy.newaxis, :])
        
        return (word_ids, word_cts)

In [291]:
parsed_corpus = parse_data(doc)
number_of_topics = 4
number_of_types = len(type_to_index)
number_of_documents = len(parsed_corpus[0])
alpha_alpha = np.zeros(number_of_topics) + 1/number_of_topics
alpha_beta = np.zeros(number_of_types) + 1/number_of_types
eta = np.random.gamma(100., 1./100., (number_of_topics,number_of_types))
gamma = numpy.zeros((number_of_documents, number_of_topics)) + alpha_alpha[numpy.newaxis, :] + 1.0 * number_of_types / number_of_topics
word_ids = parsed_corpus[0]
word_cts = parsed_corpus[1]       
number_of_documents = len(word_ids) 
hyper_parameter_optimize_interval=1

In [205]:
def compute_dirichlet_expectation(dirichlet_parameter):
    if (len(dirichlet_parameter.shape) == 1):
        return (sp.special.psi(dirichlet_parameter) -sp.special.psi(numpy.sum(dirichlet_parameter)))
    return (sp.special.psi(dirichlet_parameter) - sp.special.psi(np.sum(dirichlet_parameter, 1))[:, numpy.newaxis])

In [288]:
def e_step(parsed_corpus=None,local_parameter_iteration=50,
local_parameter_converge_threshold=1e-6):
             
    document_log_likelihood = 0
    words_log_likelihood = 0

    # initialize a V-by-K matrix phi sufficient statistics
    phi_sufficient_statistics = np.zeros((number_of_topics, number_of_types))

    # initialize a D-by-K matrix gamma values
    gamma_values = np.zeros((number_of_documents, number_of_topics)) + alpha_alpha[np.newaxis, :] + 1.0 * number_of_types / number_of_topics

    E_log_eta = compute_dirichlet_expectation(eta)
    E_log_prob_eta = E_log_eta - sp.misc.logsumexp(E_log_eta, axis=1)[:, np.newaxis]

    # iterate over all documents
    for doc_id in np.random.permutation(number_of_documents):
        # compute the total number of words
        # total_word_count = corpus[doc_id].N()
        total_word_count = np.sum(word_cts[doc_id])

        # initialize gamma for this document
        gamma_values[doc_id, :] = alpha_alpha + 1.0 * total_word_count / number_of_topics

        term_ids = word_ids[doc_id]
        term_counts = word_cts[doc_id]

        # update phi and gamma until gamma converges
        for gamma_iteration in range(local_parameter_iteration):
            log_phi = E_log_eta[:, term_ids].T + np.tile(sp.special.psi(gamma_values[doc_id, :]), (word_ids[doc_id].shape[0], 1))

            log_phi -= sp.misc.logsumexp(log_phi, axis=1)[:, np.newaxis]

            gamma_update = alpha_alpha + np.array(np.sum(np.exp(log_phi + np.log(np.repeat(term_counts, 4, axis=0).T)), axis=0))

            mean_change = np.mean(abs(gamma_update - gamma_values[doc_id, :]))
            gamma_values[doc_id, :] = gamma_update
            if mean_change <= local_parameter_converge_threshold:
                break

        # compute the alpha terms
        document_log_likelihood += sp.special.gammaln(np.sum(alpha_alpha)) - np.sum(sp.special.gammaln(alpha_alpha))
        # compute the gamma terms
        document_log_likelihood += np.sum(sp.special.gammaln(gamma_values[doc_id, :])) - sp.special.gammaln(np.sum(gamma_values[doc_id, :]))
        # compute the phi terms
        document_log_likelihood -= np.sum(np.dot(term_counts, np.exp(log_phi) * log_phi))

# compute the p(w_{dn} | z_{dn}, \eta) terms, which will be cancelled during M-step during training
        words_log_likelihood += np.sum(np.exp(log_phi.T + np.log(term_counts)) * E_log_prob_eta[:, term_ids])      
        phi_sufficient_statistics[:, term_ids] += np.exp(log_phi + np.log(term_counts.transpose())).T
        
    if parsed_corpus == None:
        gamma = gamma_values
        return (document_log_likelihood, phi_sufficient_statistics, gamma)
    else:
        return (words_log_likelihood, gamma_values)

In [301]:
def m_step(phi_sufficient_statistics):
        # Note: all terms including E_q[p(\eta|\beta)], i.e., terms involving \Psi(\eta), are cancelled due to \eta updates
            
        # compute the beta terms
        topic_log_likelihood = number_of_topics * (sp.special.gammaln(np.sum(alpha_beta)) - np.sum(sp.special.gammaln(alpha_beta)))
        # compute the eta terms
        topic_log_likelihood += np.sum(np.sum(sp.special.gammaln(eta), axis=1) - sp.special.gammaln(np.sum(eta, axis=1)))
        
        eta_temp = phi_sufficient_statistics + alpha_beta
        
        # compute the sufficient statistics for alpha and update
        alpha_sufficient_statistics = sp.special.psi(gamma) - sp.special.psi(np.sum(gamma, axis=1)[:, np.newaxis]);
        alpha_sufficient_statistics = np.sum(alpha_sufficient_statistics, axis=0);  # [np.newaxis, :];
        
        return (topic_log_likelihood, alpha_sufficient_statistics, eta_temp)

In [302]:
document_log_likelihood, phi_sufficient_statistics, gamma = e_step()
topic_log_likelihood, alpha_sufficient_statistics, eta = m_step(phi_sufficient_statistics)
joint_log_likelihood = document_log_likelihood + topic_log_likelihood

In [295]:
def optimize_hyperparameters(alpha_sufficient_statistics, hyper_parameter_iteration=100, hyper_parameter_decay_factor=0.9, hyper_parameter_maximum_decay=10,alpha=alpha_alpha, hyper_parameter_converge_threshold=1e-6):

        alpha_update = alpha;        
        decay = 0;
        for alpha_iteration in range(hyper_parameter_iteration):
            alpha_sum = np.sum(alpha)
            alpha_gradient = number_of_documents * (sp.special.psi(alpha_sum) - sp.special.psi(alpha)) + alpha_sufficient_statistics
            alpha_hessian = -number_of_documents * sp.special.polygamma(1,alpha)

            if np.any(np.isinf(alpha_gradient)) or np.any(np.isnan(alpha_gradient)):
                print ("illegal alpha gradient vector", alpha_gradient)

            sum_g_h = np.sum(alpha_gradient / alpha_hessian)
            sum_1_h = 1.0 / alpha_hessian

            z = number_of_documents * sp.special.polygamma(1, alpha_sum)
            c = sum_g_h / (1.0 / z + sum_1_h)

            # update the alpha vector
            while True:
                singular_hessian = False

                step_size = np.power(hyper_parameter_decay_factor, decay) * (alpha_gradient - c) / alpha_hessian
                
                if np.any(alpha <= step_size):
                    singular_hessian = True
                else:
                    alpha_update = alpha - step_size;
                
                if singular_hessian:
                    decay += 1;
                    if decay > hyper_parameter_maximum_decay:
                        break;
                else:
                    break;
                
            # compute the alpha sum
            # check the alpha converge criteria
            mean_change = np.mean(abs(alpha_update - alpha));
            alpha = alpha_update;
            if mean_change <= hyper_parameter_converge_threshold:
                break;

        return (alpha)

In [296]:
alpha_alpha = optimize_hyperparameters(alpha_sufficient_statistics)

In [298]:
words_log_likelihood, corpus_gamma_values = e_step(parsed_corpus = parsed_corpus)

In [304]:
E_log_eta = compute_dirichlet_expectation(eta);
for topic_index in range(number_of_topics):
    print("==========\t%d\t==========\n" % (topic_index))

    beta_probability = np.exp(E_log_eta[topic_index, :] - sp.misc.logsumexp(E_log_eta[topic_index, :]));

    i = 0;
    for type_index in reversed(np.argsort(beta_probability)):
        i += 1;
        print("%s\t%g\n" % (index_to_type[type_index], beta_probability[type_index]));

==========	0	==========

year	0.006289

peopl	0.00508852

percent	0.00492242

report	0.00427809

two	0.00417133

presid	0.00398333

new	0.00372377

nation	0.00340916

time	0.00340066

state	0.00334827

offic	0.00328299

last	0.00315835

bush	0.00310807

million	0.00297081

govern	0.0028922

first	0.00282069

expect	0.00267743

soviet	0.00267399

today	0.00266388

plan	0.00260563

go	0.00250206

compani	0.00250122

billion	0.00248521

offici	0.00248396

work	0.00246206

call	0.00244539

cent	0.00244284

market	0.00243534

forc	0.00239487

day	0.00238233

price	0.00236003

say	0.00235416

citi	0.00227516

countri	0.00223473

share	0.00214421

make	0.00213992

trade	0.00212436

bank	0.002022

american	0.00202108

congress	0.00201211

hou	0.00198549

court	0.00198223

unit	0.00197996

told	0.00188563

feder	0.00187642

public	0.00184894

rate	0.00183583

wednesday	0.0018089

south	0.00178073

get	0.00177984

depart	0.00177451

tuesday	0.00175973

back	0.00174442

home	0.00174129

member	0.